In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
products = pd.read_csv(r"C:\Users\pmoreira\Desktop\PBS\DescriptiveAnalysis\Product_info.csv")
sales = pd.read_csv(r"C:\Users\pmoreira\Desktop\PBS\DescriptiveAnalysis\Sales_info.csv")
account = pd.read_csv(r"C:\Users\pmoreira\Desktop\PBS\DescriptiveAnalysis\Account_info.csv")


In [ ]:
print(type(products))
products

In [ ]:
print(type(account))
account

In [ ]:
print(type(sales))
sales

In [ ]:
#Encrypted SKU & Product Name
sales_merged = sales.merge(products [['encrypt_sku','product_dsc', 'cat_dsc_ext']], on='encrypt_sku', how='left')
sales_merged.head(1)

In [ ]:
#Check the frequency of each product on the sales data
products_frequency = sales_merged[['encrypt_sku','product_dsc']].value_counts()
products_frequency

#Top 5 products sold
top_5_products = products_frequency.head(5)
top_5_products.plot (kind ='bar')
plt.title('Top 5 Products Sold')
plt.ylabel('Relative Frequency')
plt.xlabel('Products')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Caracterization of the Transactions
transaction_sizes = sales_merged.groupby('transaction_id')['product_dsc'].count()
transaction_sizes.describe()

In [ ]:
# Quantas transações únicas existem
num_transactions = sales_merged['transaction_id'].nunique()

# Para cada produto, contar em quantas transações ele aparece
product_transaction_counts = sales_merged.groupby('product_dsc')['transaction_id'].nunique()

# Calcular o suporte: frequência relativa de transações
product_support = product_transaction_counts / num_transactions

# Filtrar produtos com suporte >= 0.02

products_with_support = product_support[product_support >= 0.02]

print(f"Número de produtos com suporte >= 0.03: {len(products_with_support)}")
products_with_support.sort_values(ascending=False)



In [ ]:
# Encode the Transactions
transactions = sales_merged.groupby('transaction_id')['product_dsc'].apply(list).tolist()
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
transactions_matrix = te.fit_transform(transactions)
encoded_sales = pd.DataFrame(transactions_matrix, columns=te.columns_)
encoded_sales.head(1)


In [ ]:
# Run Apriori to Get Frequent Co-Purchased Products (0,2% of the transactions)
min_supp = 0.002
freq_copurchased = apriori(encoded_sales, min_support=min_supp, use_colnames=True) 
print(f"Number of Frequent Co-Purchased Products: {len(freq_copurchased)}")

In [ ]:
freq_copurchased.sort_values(by= "support",ascending=False)

In [ ]:
freq_copurchased['length'] = freq_copurchased['itemsets'].apply(len)
print(freq_copurchased['length'].value_counts())


In [ ]:
#Generate association rules from frequent itemsets
min_conf=0.1
rules = association_rules(freq_copurchased, metric="confidence", min_threshold=min_conf,num_itemsets=0.2)
rules

In [ ]:
from mlxtend.frequent_patterns import apriori
import matplotlib.pyplot as plt

# Run Apriori with lower support
min_supp = 0.002  # 0.2%
freq_copurchased = apriori(encoded_sales, min_support=min_supp, use_colnames=True)

# Add length of itemset
freq_copurchased['length'] = freq_copurchased['itemsets'].apply(lambda x: len(x))

# Filter itemsets with more than 1 item (i.e., combinations)
filtered = freq_copurchased[freq_copurchased['length'] > 1]

# Check if there are any
print(f"Número de combinações com suporte >= {min_supp}: {len(filtered)}")

if not filtered.empty:
    # Get top 10 by support
    top10 = filtered.sort_values(by='support', ascending=False).head(10)
    
    # Convert sets to strings for labeling
    top10['itemset_str'] = top10['itemsets'].apply(lambda x: ', '.join(x))

    # Plot
    plt.figure(figsize=(10, 6))
    plt.bar(top10['itemset_str'], top10['support'])
    plt.title("Top 10 Frequent Itemsets (size > 1)")
    plt.xlabel("Itemset")
    plt.ylabel("Support")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("Ainda não há itemsets de tamanho > 1 com esse suporte mínimo.")


In [ ]:
from mlxtend.frequent_patterns import association_rules

# Gerar regras
rules = association_rules(freq_copurchased, metric="confidence", min_threshold=0.2)

# Adicionar coluna com o número de itens no antecedente
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))

# Filtrar regras com pelo menos 1 item no antecedente e suporte mínimo
rules_filtered = rules[(rules['antecedent_len'] >= 1) & (rules['support'] >= 0.002)]

# Ordenar por lift e mostrar as top 10
top10_rules = rules_filtered.sort_values(by='confidence', ascending=False).head(10)

# Mostrar colunas relevantes
top10_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


In [ ]:
# Supondo que você tenha um DataFrame com as transações (ex: trans_id, cat_dsc_ext)
# Agrupar categorias por transação
basket = sales_merged.groupby(['transaction_id', 'cat_dsc_ext']).size().unstack().fillna(0)

# Converter para 1/0 (presença/ausência do item)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)


In [ ]:
from mlxtend.frequent_patterns import apriori

# Gerar itemsets frequentes com suporte mínimo
freq_items = apriori(basket, min_support=0.002, use_colnames=True)


In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(freq_items, metric="confidence", min_threshold=0.2)

In [ ]:
# Adicionar tamanho do antecedente
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))

# Filtrar regras com pelo menos 1 item no antecedente e suporte mínimo
rules_filtered = rules[(rules['antecedent_len'] >= 1) & (rules['support'] >= 0.002)]

# Ordenar pelas 10 regras com maior confiança
top10_rules = rules_filtered.sort_values(by='confidence', ascending=False).head(10)

# Mostrar como tabela limpa (sem índice)
print("\n=== ASSOCIATION RULES GENERATION ===")
print(f"Generated {len(rules_filtered)} association rules\n")
print("Top 10 rules by confidence:\n")
print(top10_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_string(index=False))



Let´s segment the customer for the strongest itemsets (there 2 only two)

In [ ]:
# ----------------------------
# 2. Merge ALL product and account columns into sales
# ----------------------------
sales_merged_account = sales_merged.merge(account, on='account_no', how='left')  # All account columns

# ----------------------------
# 3. Build transactions from ALL account characteristics per transaction
# ----------------------------

# Drop columns not useful for profiling (like IDs, dates, numbers)
account_cols = account.select_dtypes(include='object').columns.tolist()
account_cols = [col for col in account_cols if col != 'account_no']  # Remove ID column

# Fill NAs to avoid issues
sales_merged_account[account_cols] = sales_merged_account[account_cols].fillna('Missing')

# Aggregate account features by transaction
account_features = sales_merged_account.groupby('transaction_id')[account_cols].agg(lambda x: list(set(x)))

# Convert each transaction into a flat list of attribute values
transactions = account_features.apply(lambda row: sum(row.tolist(), []), axis=1).tolist()

# ----------------------------
# 4. Define WRAcc target: 1 if flour + sugar are both bought
# ----------------------------
product_groups = sales_merged_account.groupby('transaction_id')['product_dsc'].apply(list)
'''
transaction_targets = product_groups.apply(lambda items: int(
    'FAR TRIGO C/FER SUP FINA CONTINENTE 1KG' in items and
    'AÇÚCAR BRANCO CONTINENTE 1KG' in items)).reset_index(drop=True)

# ----------------------------
# 5. Encode account characteristics using TransactionEncoder
# ----------------------------
te = TransactionEncoder()
transactions_matrix = te.fit_transform(transactions)
encoded_sales = pd.DataFrame(transactions_matrix, columns=te.columns_)

# ----------------------------
# 6. Run Apriori (min support ≥ 0.2%)
# ----------------------------
min_support = 0.002
frequent_itemsets = apriori(encoded_sales, min_support=min_support, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# ----------------------------
# 7. Compute WRAcc
# ----------------------------
def compute_wracc(itemset, df, y):
    if not itemset:
        return 0
    cover = df[list(itemset)].all(axis=1)
    if cover.sum() == 0:
        return 0
    p_s = y[cover].mean()
    p_d = y.mean()
    return cover.mean() * (p_s - p_d)

frequent_itemsets['WRAcc'] = frequent_itemsets['itemsets'].apply(
    lambda s: compute_wracc(s, encoded_sales, transaction_targets)
)

# ----------------------------
# 8. Show top itemsets by WRAcc
# ----------------------------
filtered = frequent_itemsets[frequent_itemsets['length'] >= 1]
print(f"\nNúmero de combinações com suporte >= {min_support}: {len(filtered)}")

if not filtered.empty:
    top10 = filtered.sort_values(by='WRAcc', ascending=False).head(20)
    print("\nTop 10 Account Characteristic Combinations by WRAcc (length >= 1):\n")
    print(top10[['itemsets', 'support', 'WRAcc']].to_string(index=False))
else:
    print("Ainda não há combinações de características de conta com esse suporte mínimo.")
'''
# ----------------------------Agora testar para outro conjunto de dados----------------------------

transaction_targets = product_groups.apply(lambda items: int(
    'PERNA DE FRANGO (2KG) COMPRA/AT' in items and
    'PEITO DE FRANGO (2KG) COMPRA/AT' in items)).reset_index(drop=True)

# ----------------------------
# 5. Encode account characteristics using TransactionEncoder
# ----------------------------
te = TransactionEncoder()
transactions_matrix = te.fit_transform(transactions)
encoded_sales = pd.DataFrame(transactions_matrix, columns=te.columns_)

# ----------------------------
# 6. Run Apriori (min support ≥ 0.2%)
# ----------------------------
min_support = 0.002
frequent_itemsets = apriori(encoded_sales, min_support=min_support, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# ----------------------------
# 7. Compute WRAcc
# ----------------------------
def compute_wracc(itemset, df, y):
    if not itemset:
        return 0
    cover = df[list(itemset)].all(axis=1)
    if cover.sum() == 0:
        return 0
    p_s = y[cover].mean()
    p_d = y.mean()
    return cover.mean() * (p_s - p_d)

frequent_itemsets['WRAcc'] = frequent_itemsets['itemsets'].apply(
    lambda s: compute_wracc(s, encoded_sales, transaction_targets)
)

# ----------------------------
# 8. Show top itemsets by WRAcc
# ----------------------------
filtered = frequent_itemsets[frequent_itemsets['length'] >= 1]
print(f"\nNúmero de combinações com suporte >= {min_support}: {len(filtered)}")

if not filtered.empty:
    top10 = filtered.sort_values(by='WRAcc', ascending=False).head(20)
    print("\nTop 10 Account Characteristic Combinations by WRAcc (length >= 1):\n")
    print(top10[['itemsets', 'support', 'WRAcc']].to_string(index=False))
else:
    print("Ainda não há combinações de características de conta com esse suporte mínimo.")

